<a href="https://colab.research.google.com/github/maab2198/applied_ds_python/blob/master/C1_M1A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = "drive/My Drive/Colab Notebooks/C1_M1Files/"

# Assignment 4 - Hypothesis Testing

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. 

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.


Data files:
* [Zillow research data site](http://www.zillow.com/research/data/) -  housing data for the United States. ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) list was copy and pasted into the file ```university_towns.txt```.
* [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.



In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

#University towns

In [0]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_table(path + "university_towns.txt", names =["Region"])
    
    df["State"] = df[df['Region'].str.contains("edit")]
    df["RegionName"]= df[df['Region'].str.contains("edit") == False]["Region"].apply( lambda x: str(x).split('(')[0].strip())
    df["State"].fillna( method ='ffill', inplace = True)
    df["State"] = df["State"].apply(lambda x: str(x).split("[", maxsplit=1)[0])
    df = df.dropna()
    return df[["State","RegionName"]].reset_index(drop=True)
              
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


#Test

---



In [0]:
import re
import pandas as pd
import numpy as np
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))


Shape test:  Passed
Index test:  Passed
Column test:  Passed
\n test:  Passed
Trailing whitespace test: Passed
"(" test: Passed
"[" test: Passed
State test:  Passed
RegionName test:  Passed


#GDP

In [0]:
gdp = pd.read_excel(path + 'gdplev.xls', skiprows=7) 

gdp = gdp.drop(gdp.columns[[0,1,2,3,5,7]],axis =1 )
gdp.columns = ["Q","GDP 2009"]
gdp = gdp[gdp["Q"] >"2000q1"].reset_index(drop=True)
gdp

,Q,GDP 2009
0,2000q2,12592.5
1,2000q3,12607.7
2,2000q4,12679.3
3,2001q1,12643.3
4,2001q2,12710.3
...,...,...
60,2015q2,16374.2
61,2015q3,16454.9
62,2015q4,16490.7
63,2016q1,16525.0


In [0]:
# calc diff
gdp["down"] = ((gdp['GDP 2009'] -  gdp['GDP 2009'].shift(+1)) < 0)
# find recession bottom
for index in range(2, len(gdp)-2):
  gdp.loc[index,"bottom"] = (gdp.loc[index,"down"]  and gdp.loc[index-1,"down"] and gdp.loc[index-2,"down"]and (not(gdp.loc[index +1,"down"])) and(not(gdp.loc[index+2,"down"])))
gdp


,Q,GDP 2009,down,bottom,RecessionStart
0,2000q2,12592.5,False,NaN,False
1,2000q3,12607.7,False,NaN,False
2,2000q4,12679.3,False,False,False
3,2001q1,12643.3,True,False,False
4,2001q2,12710.3,False,False,False
...,...,...,...,...,...
60,2015q2,16374.2,False,False,False
61,2015q3,16454.9,False,False,False
62,2015q4,16490.7,False,False,False
63,2016q1,16525.0,False,NaN,False


In [0]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp['RecessionStart'] = (gdp['down']) & (gdp['down'].shift(-1)) & (gdp['down'].shift(-2)) 
    return gdp[gdp['RecessionStart']==True].iloc[0,0]

get_recession_start()

'2008q3'

In [0]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return gdp[gdp['bottom'] == True].iloc[0,0]
    
get_recession_bottom()

'2009q2'

In [0]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    end = gdp[gdp['bottom'].shift(+2) == True]
    return end.iloc[0,0]
    
get_recession_end()

'2009q4'

#convert_housing_data_to_quarters

In [0]:
def convert_housing_data_to_quarters():
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    df = pd.read_csv( path + "City_Zhvi_AllHomes.csv" , index_col = 2)
    df = df.rename(index = states)
    df = df.reset_index()

    #data from 2000q1
    cols = list(filter(lambda x: (str(x[0]).isdigit() and x[:4] >= "2000"),list(df.columns)))

    #resample time-series data by quarters
    test = df[cols].T
    test.index = pd.to_datetime(test.index)
    test = test.resample("Q").mean()
    
    #2000-09-30 => 2000q3
    arr = [str(el)[:4] + "q" + str(int(int(str(el)[5:7])/3)) for el in test.index] 
  
    copy = df[["State","RegionName"]].copy()
    copy[arr] = test.copy().T
    return copy.set_index(["State","RegionName"])
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,2003q3,2003q4,2004q1,2004q2,2004q3,2004q4,2005q1,2005q2,2005q3,2005q4,2006q1,2006q2,2006q3,2006q4,2007q1,2007q2,2007q3,2007q4,2008q1,2008q2,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4,2010q1,2010q2,2010q3,2010q4,2011q1,2011q2,2011q3,2011q4,2012q1,2012q2,2012q3,2012q4,2013q1,2013q2,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435650.000000,432133.333333,422666.666667,431800.000000,438766.666667,464500.000000,488133.333333,513700.000000,520033.333333,524700.000000,517233.333333,514666.666667,510200.000000,513366.666667,508500.000000,503933.333333,499766.666667,487933.333333,477733.333333,465833.333333,455933.333333,458366.666667,469700.000000,476933.333333,478566.666667,477266.666667,476700.000000,472133.333333,469200.000000,463700.000000,460200.000000,462700.000000,467233.333333,469166.666667,469033.333333,472233.333333,481366.666667,492433.333333,503133.333333,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,287000.000000,302233.333333,318900.000000,334500.000000,350633.333333,370833.333333,395233.333333,425566.666667,456166.666667,477366.666667,495066.666667,518400.000000,545566.666667,571066.666667,581766.666667,587500.000000,592966.666667,592533.333333,591766.666667,589500.000000,577300.000000,561366.666667,535300.000000,502266.666667,469500.000000,443966.666667,426266.666667,413900.000000,406366.666667,404333.333333,413366.666667,411166.666667,406766.666667,402066.666667,397100.000000,390100.000000,382100.000000,375566.666667,370600.000000,375333.333333,385566.666667,396566.666667,411533.333333,432866.666667,455066.666667,471933.333333,484266.666667,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,178966.666667,183700.000000,187100.000000,194300.000000,202066.666667,204133.333333,206200.000000,208600.000000,212766.666667,218333.333333,223900.000000,228533.333333,233166.666667,237500.000000,242400.000000,245200.000000,245700.000000,246700.000000,247233.333333,246400.000000,246266.666667,246833.333333,243733.333333,237900.000000,232000.000000,227033.333333,223766.666667,219700.000000,214100.000000,211666.666667,207300.000000,203800.000000,198933.333333,191366.666667,187400.000000,181600.000000,174266.666667,170200.000000,166933.333333,167033.333333,168133.333333,167233.333333,168366.666667,172633.333333,180800.000000,185866.666667,190900.000000,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,62166.666667,63500.000000,64733.333333,66200.000000,68466.666667,70733.333333,73733.333333,76733.333333,80566.666667,85533.333333,89700.000000,94866.666667,100166.666667,104900.000000,108333.333333,112366.666667,115466.666667,116633.333333,117333.333333,118700.000000,120833.333333,120800.000000,119566.666667,118133.333333,116933.333333,115866.666667,116200.000000,116166.666667,116733.333333,118566.666667,121166.666667,122800.000000,120166.666667,117766.666667,117100.000000,116300.000000,115466.666667,114700.000000,112733.333333,110533.333333,109933.333333,110833.333333,112000.000000,111733.333333,111000.0

#TTest

In [0]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    univ_towns = get_list_of_university_towns().set_index(["State","RegionName"])
    housing_data = convert_housing_data_to_quarters()

    # bwfore recession start and the recession bottom points
    qrt_bfr_rec_start_id =  housing_data.columns.get_loc(get_recession_start()) - 1
    qrt_bfr_rec_start = housing_data.iloc[:,qrt_bfr_rec_start_id].name
    qrt_rec_bottom = get_recession_bottom()

    rec_cols = [qrt_bfr_rec_start,qrt_rec_bottom]
    housing_data = housing_data[rec_cols]

    #PriceRatio 
    housing_data['PriceRatio'] = housing_data[qrt_bfr_rec_start].div(housing_data[qrt_rec_bottom])

    #Split all data into univ towns and not univ towns
    univ = pd.merge(univ_towns, housing_data, how='inner', left_index=True, right_index=True)
    non_univ = housing_data.loc[~(housing_data.index.isin(univ.index))]

    #better = "university town" or "non-university town" based on which group has the lowest mean
    better = "university town" if  univ['PriceRatio'].mean() < non_univ['PriceRatio'].mean()  else "non-university town" 

    pvalue = stats.ttest_ind(non_univ['PriceRatio'].dropna(), univ['PriceRatio'].dropna()).pvalue
    return (pvalue < 0.01, pvalue, better)
run_ttest()


(True, 0.002724063704761164, 'university town')

In [0]:
# test output type (different, p, better)
def test_q6():
    q6 = run_ttest()
    different, p, better = q6

    res = 'Type test: '
    res += ['Failed\n','Passed\n'][type(q6) == tuple]

    res += 'Test "different" type: '
    res += ['Failed\n','Passed\n'][type(different) == bool or type(different) == np.bool_]

    res += 'Test "p" type: '
    res += ['Failed\n','Passed\n'][type(p) == np.float64]

    res +='Test "better" type: '
    res += ['Failed\n','Passed\n'][type(better) == str]
    if type(better) != str:
        res +='"better" should be a string with value "university town" or  "non-university town"'
        return res
    res += 'Test "different" spelling: '
    res += ['Failed\n','Passed\n'][better in ["university town", "non-university town"]]
    return res
print(test_q6())

Type test: Passed
Test "different" type: Passed
Test "p" type: Passed
Test "better" type: Passed
Test "different" spelling: Passed

